$$ Z(M) = x^r\cdot \hat{Q}_M(x^{-2})$$

$$Z(M\setminus e) = Z(M/e) + Z(M) + \sum_{G > e; e\notin coloops(M\mid_G)}{Z(M/G)\cdot\tau(M^G_e)}$$

In [699]:
import sys
sys.path.append('..')
from db1 import *

R = PolynomialRing(ZZ, 't')
t = R.gen()

def is_paving(M):
    n = M.size()
    r = M.rank()
    return (len(M.independent_r_sets(r-1)) == binomial(n, r-1))

def q_kl(k, h):
    return kazhdan_lusztig_inverse_uniform(k, h+1) - kazhdan_lusztig_inverse_uniform(k-1, h)

def kl_inverse_fast(M):
    if M.loops(): return R(0)
    k, n = M.rank(), M.size()
    if k == n or k == 0: return R(1)
    if not M.is_connected():
        ans = R(1)
        CC = M.components()
        for N in CC:
            res = M.delete(M.groundset() - N)
            ans = ans * kl_inverse_fast(res)
        return ans

    if is_paving(M):
        return kl_inverse_paving(M)
    if is_paving(M.dual()):
        return kl_inverse_copaving(M)
        
    LF = M.lattice_of_flats()
    ans = R(0)
    for F in LF:
        if len(F) != n:
            Res = M.delete(M.groundset() - F)
            Con = M.contract(F)
            chi = characteristic_polynomial(Con)(1/t) * t**(Con.rank())
            PPP = kl_inverse_fast(Res)(t) * (-1)**(Res.rank())
            ans = ans + chi * PPP
    assert (t**k * ans(1/t)).numerator() == -ans(t)
    ans = ans.numerator() * (-1)**(k+1)
    return ans.truncate((k+1)//2)

def kazhdan_lusztig_inverse_uniform(k, n):
    if k == n:
        return R(1)
    d = k
    m = n - d
    ans = 0
    for j in range((d-1)//2 + 1):
        ans = ans + m * (d-2*j)/((m+j) * (m+d-j)) * binomial(d, j) * t**j
    return ans * binomial(m+d, d)

def kl_inverse_paving(M):
    assert is_paving(M)
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.hyperplanes():
        h = len(H)
        if h >= k:
            ans = ans - q_kl(k, h)
    return ans

def kl_inverse_copaving(M):
    assert is_paving(M.dual())
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.dual().hyperplanes():
        h = len(H)
        if h >= n-k:
            ans = ans - kli_vtilde_dual(n-k, h, n) + kazhdan_lusztig_inverse_uniform(h-n+k+1, h) * kazhdan_lusztig_inverse_uniform(n-h-1, n-h)
    return ans

def kli_vtilde_dual(k, h, n):
    return helper1(n-k, h, n)

def helper1(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_inverse_uniform(k, n)
    ans2 = helper2(c, k, n)
    ans3 = kazhdan_lusztig_inverse_uniform(k-c+1, h) * kazhdan_lusztig_inverse_uniform(c-1, c)
    return ans1 - ans2 + ans3

def helper2(c, k, n):
    h = n - c
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * (-1)**(c-1+j) * kazhdan_lusztig_inverse_uniform(c-1, c) * t**(k-c-j+1) * chuly(k-c-j+1, n-c-j)(1/t)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(i+j) * t**(k-i-j) * helper4(c, k, n, i, j)(1/t)
    ans = ans.numerator().truncate((k-1)//2 + 1)
    if ans[0] < 0:
        ans = -ans
    return ans

def helper3(c, k, n):
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * kazhdan_lusztig_uniform_matroid(c-1, c) * (-1)**(k-c-j+1) * kazhdan_lusztig_inverse_uniform(k-c-j+1, n-c-j)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(k-i-j) * helper2(c-i, k-i-j, n-i-j)
    return -ans

def helper4(c, k, n, i, j):
    ans = 0
    for l in range(c-i-1):
        ans = ans + (-1)**l * (t-1)**(max(n-i-j-l-1, 0))
    for u in range(n-k-1):
        ans = doit_once(ans)
    return ans

def chuly(a, b):
    ans = (t-1)**b
    for i in range(b-a):
        ans = doit_once(ans)
    return ans

def doit_once(p):
    p = p // t**2
    p = p * t
    p = p - p(1)
    return p

def lorenzo(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_uniform_matroid(k, n) + kazhdan_lusztig_uniform_matroid(k-c+1, h) * kazhdan_lusztig_uniform_matroid(c-1, c)
    ans2 = helper3(c, k, n)
    return ans1 - ans2

In [700]:
P = LaurentPolynomialRing(ZZ, 'x')

def kl(M):
    return M.lattice_of_flats().kazhdan_lusztig_polynomial()(x)

invkl = kl_inverse_fast

def tau(matroid):
    print(matroid)
    if matroid.rank() % 2 == 0:
        print("even")
        return 0
    print(kl(matroid))
    klm = kl(matroid).monomial_coefficients()
    return klm.get(x^matroid.rank()//2, 0)

def qhat(M):
    return (-1) ^ M.rank() * kl_inverse_fast(M)

def the_set_S(flats, e):
    return set(F for F in flats if e.isdisjoint(F) and F.union(e) in flats)

In [701]:
def A(L):
    return (x^L.rank() * qhat(L)(x^-2)).expand()

def delform_rhs(M, e):
    r = M.rank()
    x = P.gen()
    rhs = (x + x^-1) * A(M.contract(e)) + A(M)
    print(M, rhs.expand())
    for G in L:
        if e in G:
            restricted = M.delete(M.groundset() - G)
            if e not in restricted.coloops():
                print(G, restricted)
                tauu = tau(restricted.contract(e))
                Am = A(M.contract(G))
                if tauu != 0:
                    print(f"Flat: {G}, A(M.contract(G)): {A(M.contract(G)).expand()}, "
                        f"KL(restricted.contract(e)): {kl(restricted.contract(e))}, Tau: {tauu}")
                rhs = rhs + Am * tauu
    return rhs

In [702]:
M = matroids.CompleteGraphic(6)
e = next(x for x in M.groundset() if x not in M.coloops())
L = M.lattice_of_flats()

In [703]:
[A(M.delete(e)), A(M), ((x + x^-1) * A(M.contract(e))).expand()]

[-96*x^5 - 76*x^3 - 13*x, -120*x^5 - 86*x^3 - 15*x, 24*x^5 + 34*x^3 + 10*x]

In [704]:
delform_rhs(M, e).expand()

M(K6): Graphic matroid of rank 5 on 15 elements -96*x^5 - 52*x^3 - 5*x
frozenset({0, 1, 5}) Graphic matroid of rank 2 on 3 elements
Graphic matroid of rank 1 on 2 elements
1


AttributeError: 'sage.symbolic.expression.Expression' object has no attribute 'monomial_coefficients'